In [3]:
import boto3
# Retrieve the list of existing buckets
s3 = boto3.client('s3')
response = s3.list_buckets()

# Output the bucket names
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

Existing buckets:
  aws-emr-resources-235671948910-us-east-1
  aws-logs-235671948910-us-east-1
  privacylost
  zhan-commoncrawl


In [4]:
# Retrieve a bucket's ACL
result = s3.get_bucket_acl(Bucket='commoncrawl')
print(result)

{'ResponseMetadata': {'RequestId': 'VB23KF97P30D9EJM', 'HostId': '7FSAE8dT/3P6DkgnqgU5PfjqGKDuNmM5dvEy5JuNAgwiGq//NjjBfFvrRaxDnrxToKYB3yD5hsU=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': '7FSAE8dT/3P6DkgnqgU5PfjqGKDuNmM5dvEy5JuNAgwiGq//NjjBfFvrRaxDnrxToKYB3yD5hsU=', 'x-amz-request-id': 'VB23KF97P30D9EJM', 'date': 'Sun, 08 Jan 2023 09:23:49 GMT', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'Owner': {'DisplayName': 'sara', 'ID': 'fc504895e3c44cbe2a063212bf4f392a0c24a398910f6f535ec911c404ee8347'}, 'Grants': [{'Grantee': {'DisplayName': 'sara', 'ID': 'fc504895e3c44cbe2a063212bf4f392a0c24a398910f6f535ec911c404ee8347', 'Type': 'CanonicalUser'}, 'Permission': 'FULL_CONTROL'}]}


In [5]:
file_name = 'crawl-data/CC-MAIN-2015-27/segments/1435375096870.66/warc/CC-MAIN-20150627031816-00093-ip-10-179-60-89.ec2.internal.warc.gz'

offset = 409791388
length = 12641
offset_end = offset + length - 1
byte_range = "bytes={offset}-{end}".format(offset=offset, end=offset_end)
resp = s3.get_object(Bucket="commoncrawl", Key=file_name, Range=byte_range)["Body"]

ClientError: An error occurred (SlowDown) when calling the GetObject operation (reached max retries: 4): Please reduce your request rate.

In [8]:
from warcio.archiveiterator import ArchiveIterator
for record in ArchiveIterator(resp):
    url = record.rec_headers.get_header("WARC-Target-URI")
    text = record.content_stream().read()
    print(text)

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html  xmlns="http://www.w3.org/1999/xhtml" xml:lang="gl" lang="gl">\n<head>\t<title>Portada - CMATI</title> \n<meta content="text/html; charset=UTF-8" http-equiv="content-type" />\n<link rel="Shortcut Icon" href="/tema-portal-cmati/images/liferay.ico" />\n<link href="/html/portal/css.jsp?browserId=other&amp;themeId=temaportadacmati_WAR_temaportalcmati&amp;colorSchemeId=01&amp;minifierType=css&amp;t=1348207760000" rel="stylesheet" type="text/css" />\n<script type="text/javascript">/*<![CDATA[*/var Liferay={Browser:{acceptsGzip:function(){return true},getMajorVersion:function(){return 0},getRevision:function(){return""},getVersion:function(){return""},isAir:function(){return false},isChrome:function(){return false},isFirefox:function(){return false},isGecko:function(){return false},isIe:function(){return false},isIphone:function(){return false},isLinux:function(){

## analyse dataset

In [4]:
import pandas as pd
from ast import literal_eval 

# df = pd.read_csv("../parsed_cc_results_10.txt",'\t',names = ['url','cc_path'],converters={"cc_path": literal_eval})

df = pd.read_csv("../parsed_cc_results_10.txt",names = ['url','cc_path','year','offset','length'])

In [5]:
df.head()

,url,cc_path,year,offset,length
0,www.izoa.fr,crawl-data/CC-MAIN-2021-49/segments/1637964362...,2021,896784718,62792
1,www.izoa.fr,crawl-data/CC-MAIN-2016-50/segments/1480698542...,2016,529101936,27042
2,www.izoa.fr,crawl-data/CC-MAIN-2014-41/segments/1410657137...,2014,591070386,30082
3,www.izoa.fr,crawl-data/CC-MAIN-2019-51/segments/1575540551...,2019,738781671,20978
4,www.izoa.fr,crawl-data/CC-MAIN-2015-35/segments/1440644064...,2015,505830628,22798
